# 自定义输出
· 输出函数参数
· 输出json
· 输出list
· 输出日期

In [8]:
# 讲笑话机器人：希望每次根据指令，可以输出这样的笑话（小明是怎么死的？笨死的）
from langchain_community.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator
from typing import List
import os

DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

llm = ChatOpenAI(
    model="qwen-plus",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=DASHSCOPE_API_KEY,
    temperature=0,
    max_tokens=256,
)

# 定义一个数据模型，用来描述最终的实例结构
class Joke(BaseModel):
    setup: str = Field(description="设置笑话的问题")
    punchline: str = Field(description="回答笑话的答案")

    # 验证问题是否符合要求
    @validator("setup")
    def question_mark(cls, field):
        if field[-1] not in ["?", "？"]:
            raise ValueError("问题必须以中文问号或英文问号结尾")
        return field
    
    # 添加兼容 Pydantic v1 的 schema 方法
    @classmethod
    def model_json_schema(cls, **kwargs):
        return cls.schema(**kwargs)

# 将Joke数据模型传入
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="回答用户的输入,\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt_and_model = prompt | llm
output = prompt_and_model.invoke({"query": "给我讲一个笑话"})
print(output)
parser.invoke(output)

content='```json\n{\n  "setup": "为什么程序员总是喜欢用 iPhone？",\n  "punchline": "因为他们不喜欢 Androids（安卓）！"\n}\n```' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 201, 'total_tokens': 236, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run--528b5654-4328-49f6-89b0-705d976f4e97-0'


Joke(setup='为什么程序员总是喜欢用 iPhone？', punchline='因为他们不喜欢 Androids（安卓）！')

In [10]:
# LLM的输出格式化为python list格式，类似['a', 'b', 'c']
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
import os

DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

llm = ChatOpenAI(
    model="qwen-plus",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=DASHSCOPE_API_KEY,
    temperature=0,
    max_tokens=256,
)

parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template="列出5个{subject},\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

input = prompt.format(subject="有名的中国人")
output = llm.invoke(input)
print(output)

# 格式化
parser.invoke(output)

content='马云,李娜,姚明,刘慈欣,杨利伟' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 42, 'total_tokens': 56, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run--b496acd6-915f-45aa-a6f6-08d35fb161f0-0'


['马云', '李娜', '姚明', '刘慈欣', '杨利伟']